In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

import time
import re

from bs4 import BeautifulSoup

In [3]:
def make_driver():
    opts = Options()
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument("accept-language=ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7")
    opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/124.0.0.0 Safari/537.36")
    driver = uc.Chrome(options=opts)
    return driver

In [47]:
data_path = "C:\Potenup\Drug-Detection-Chatbot\data\단일 경구약제_5,000종 리스트.xlsx"

In [66]:
df = pd.read_excel(data_path, sheet_name="1000+정보")

In [64]:
df = pd.read_excel(data_path, sheet_name="4000+정보")

In [68]:
col = ['C-\nCode', '구분', '제품명', '복지부 \n분류코드']

In [69]:
df = df[col]

In [81]:
df = df.dropna()

In [82]:
df.head()

,C-\nCode,구분,제품명,복지부 \n분류코드
0,K-000059,일반의약품,게루삼정 200T,234.0
1,K-000069,일반의약품,지엘타이밍정 12T,115.0
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0
3,K-000112,일반의약품,아로나민골드,316.0
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0


In [94]:
def cleaned_name(name):
    s = re.sub(r"\([^)]*\)", "", name)
    s = re.sub(r"\s*\d+\s*[A-Za-z가-힣]*", "", s)
    s = re.sub(r"[-+*/.,:;\s]", "", s)
    s = s.strip()

    return s

In [106]:
def find_volume(name):
    pattern = re.compile(r"\d+\s*[A-Za-z가-힣]+")
    match = pattern.search(name)
    if match:
        return match.group()
    else:
        return ''

In [107]:
for i, data in df.iterrows():
    code, _, name, _ = data
    print(code, name, ", 정제 : ", cleaned_name(name), "용법 : ", find_volume(name))

K-000059 게루삼정 200T , 정제 :  게루삼정 용법 :  200T
K-000069 지엘타이밍정 12T , 정제 :  지엘타이밍정 용법 :  12T
K-000080 마이암부톨제피정400mg , 정제 :  마이암부톨제피정 용법 :  400mg
K-000112 아로나민골드 , 정제 :  아로나민골드 용법 :  
K-000114 셉트린정 480MG(삼일) , 정제 :  셉트린정 용법 :  480MG
K-000115 자이로릭정 100MG (삼일) , 정제 :  자이로릭정 용법 :  100MG
K-000118 다오닐정5mg , 정제 :  다오닐정 용법 :  5mg
K-000121 보나링에이정 (일양) , 정제 :  보나링에이정 용법 :  
K-000122 에나폰정 10mg , 정제 :  에나폰정 용법 :  10mg
K-000147 키모랄에스정 (에이프로젠) , 정제 :  키모랄에스정 용법 :  
K-000168 후릭스정40mg , 정제 :  후릭스정 용법 :  40mg
K-000212 리단정300mg , 정제 :  리단정 용법 :  300mg
K-000250 마그밀정 500mg(삼남) , 정제 :  마그밀정 용법 :  500mg
K-000272 낙센정250mg 10T , 정제 :  낙센정 용법 :  250mg
K-000317 구주스피로닥톤정25mg , 정제 :  구주스피로닥톤정 용법 :  25mg
K-000364 페리나정 25mg 10T , 정제 :  페리나정 용법 :  25mg
K-000368 로프민캡슐(지사제)(비급여) 10C , 정제 :  로프민캡슐 용법 :  10C
K-000372 세가톤트로키정 12T , 정제 :  세가톤트로키정 용법 :  12T
K-000400 후시딘정250mg , 정제 :  후시딘정 용법 :  250mg
K-000438 네오메디코푸정 , 정제 :  네오메디코푸정 용법 :  
K-000465 피라진아미드정 500MG (유한) , 정제 :  피라진아미드정 용법 :  500MG
K-000479 노이시린에이정 500T , 정제 :  노이시

In [ ]:
main_url = "https://www.health.kr/main.asp"

In [5]:
driver = make_driver()

In [6]:
driver.get(main_url)

In [32]:
input_box_css = "#search_word"
input_box = driver.find_element(By.CSS_SELECTOR, input_box_css)
input_box.clear()
input_box.send_keys(keyword)
input_box.send_keys(Keys.ENTER)

In [33]:
html = driver.page_source

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
tds_with_onclick = [
    td for td in soup.find_all("td", class_="txtL") if td.has_attr("onclick")
]

pattern = re.compile(r"\('([^']+)'\)")  # 작은따옴표 안의 문자열만 추출
for td in tds_with_onclick:
    onclick_val = td["onclick"]
    match = pattern.search(onclick_val)
    if match:
        param = match.group(1)
        print(td.text, param)

엔비젯정10/10mg 2021021900249
엔비젯정10/20mg 2021021900250
엔비젯정10/40mg 2021021900251


In [ ]:
result_drug_base_url = "https://www.health.kr/searchDrug/result_drug.asp?drug_cd="
result_take_base_url = "https://www.health.kr/searchDrug/result_take.asp?drug_cd="
result_sunb_base_url = "https://www.health.kr/searchDrug/result_sunb.asp?drug_cd="
result_interaction_base_url = "https://www.health.kr/searchDrug/result_interaction.asp?drug_cd="
id = "2021021900249"